In [2]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [7]:
# 신경망 모델

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# L1 Conv shape=(?, 28, 28, 32)
#    Pool     ->(?, 14, 14, 32)

# 필터 개수 32개 3*3 커널 크기 
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))

# padding = 'same' 은 커널 슬라이딩시 최외각에서 한칸 밖으로 더 움직이는 옵션
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
print(L1) # shape=(?, 28, 28, 32)
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# ksize = [1,2,2,1] 이란 뜻은 2칸씩 이동하면서 출력결과를 1개 만들어 낸다는 것이다.

print(L1) #  shape=(?, 14, 14, 32)


# L1 = tf.nn.dropout(L1, keep_prob)

# L2 Conv shape=(?, 14, 14, 64)
#    Pool     ->(?, 7, 7, 64)

# 32 는 L1에서 출력된 W1의 마지막 차원, 필터의 크기이다.
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

print(L2)
# L2 = tf.nn.dropout(L2, keep_prob)

# FC 레이어: 입력값 7x7x64 -> 출력값 256
# Full connect를 위해 직전의 Pool 사이즈인 (?, 7, 7, 64) 를 참고하여 차원을 줄여줍니다.

W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])

L3 = tf.matmul(L3, W3) # --
L3 = tf.nn.relu(L3)
print(L3)

L3 = tf.nn.dropout(L3, keep_prob)
print(L3)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

print('----')

Tensor("Conv2D_8:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool_8:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("MaxPool_9:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Relu_14:0", shape=(?, 256), dtype=float32)
Tensor("dropout_4/mul:0", shape=(?, 256), dtype=float32)
----


In [3]:
# 학습

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

Epoch: 0001 Avg. cost = 0.342
Epoch: 0002 Avg. cost = 0.107
Epoch: 0003 Avg. cost = 0.078
Epoch: 0004 Avg. cost = 0.058
Epoch: 0005 Avg. cost = 0.049
Epoch: 0006 Avg. cost = 0.040
Epoch: 0007 Avg. cost = 0.036
Epoch: 0008 Avg. cost = 0.031
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.024
Epoch: 0011 Avg. cost = 0.021
Epoch: 0012 Avg. cost = 0.020
Epoch: 0013 Avg. cost = 0.017
Epoch: 0014 Avg. cost = 0.016
Epoch: 0015 Avg. cost = 0.015


In [4]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   keep_prob: 1}))

정확도: 0.9903
